Compare Jpred Training data to Alphafold:
1) Convert SCOPe ID to PDB ID 
2) Convert PDB ID to UniProt ID
3) Search Alphafold using UniProt ID
4) Save AF predictions
5) Run DSSP on Alphafold to obtain secondary structure
6) Merge all into one file and save
<!-- 6) Reduce 8 state to 3 state -->
<!-- 7) Align them?? -->

In [1]:
import pandas as pd
import glob
import numpy as np
import os
from os import path

import csv
import re
import pickle

import logging
import traceback
import requests

In [2]:
# os.chdir('/cluster/gjb_lab/2524591/data/Alphafold')
os.chdir('/Users/Sophia/Dundee/dundee/data/AF')

Obtain 1348 SCOPe IDs (got them from Jpred website) -- retr231/training

In [17]:
os.chdir('/Users/Sophia/Dundee/dundee/retr231/training')

x = glob.glob('*.fasta')
h = []

for file in x:
    scop = file[:-6]
    pdb = scop[1:5]
    y = open(file, 'r')
    a = y.read()

    df = pd.DataFrame([x.split(';') for x in a.split('\n')])
    df = df.iloc[:, 0]
    df = df[0][1:]
    h.append(df)

df2 = pd.DataFrame(h)
df2.to_csv('/Users/Sophia/Dundee/dundee/data/AF/AF_initial/1348_scope.csv')

df2


,0
0,d2ed6a1
1,d4fyxb2
2,d2cb8a_
3,d1in0a1
4,d1utga_
...,...
1343,d3b0ga1
1344,d4doia_
1345,d3rofa_
1346,d1sg4a1


,SCOPeURL
0,d1a12a_
1,d1a1xa_
2,d1a62a1
3,d1a9xa1
4,d1acoa1
...,...
1343,e1n13.1B
1344,e1qd6.1C
1345,e1qqp.12
1346,e1qtn.1A


In [3]:
os.chdir('/Users/Sophia/Dundee/dundee/data/AF/AF_initial')

In [5]:
df2 = pd.read_csv('/Users/Sophia/Dundee/dundee/data/AF/AF_initial/1348_scope.csv')


Obtain PDB IDs from SCOPe (some scope ids have the same pdb id -- not a 1-1 mapping) -- 1348 -> 1235

In [6]:
x = []
with open('1348_scope.csv', 'r') as file:
    data_read = csv.reader(file)
    for i,row in df2.iterrows():
        pdb_id = row[0][1:5]
        # print(pdb_id)
        x.append(pdb_id)

df = pd.DataFrame(x)
df.columns = ['PDB_ID']
df.to_csv('1348_pdbid.csv')
 

df_both = pd.concat([df2, df], axis=1)
df_both.to_csv('1348_pdbid+scop.csv')
df_both

# Replace some deprecated pdb id's
df_1 = df_both.replace('1ti6', '4v4e')
df_2 = df_1.replace('2o37' '4rwu')
final = df_2.replace('3arc', '3wu2')

final.to_csv('1348_pdb+scop_depr.csv')

In [7]:
# Function to save different versions of the same pdb's:

def check_file(filePath):
    if path.exists(filePath):
        numb = 1
        while True:
            newPath = "{0}_{2}{1}".format(*path.splitext(filePath) + (numb,))
            if path.exists(newPath):
                numb += 1
            else:
                return newPath
    return filePath

To convert pdb id's to Uniprot id's -- used Uniprot website

Obtain Alphafold URLs

In [11]:
# Add deprecated
os.chdir('/Users/Sophia/Dundee/dundee/data/AF')

# Obtained uniprot-pdb_aug manually -- use Uniprot website to map pdb id's to uniprot and download
uniprot = pd.read_csv('AF_initial/uniprot-pdb_aug.csv')


for i,row in uniprot.iterrows():
    isna = pd.Series(row.Uniprot).isna()[0]
    if not isna and re.compile('^[A-Z](\d+)').search(row.Uniprot):
        url = 'https://alphafold.ebi.ac.uk/files/AF-%s-F1-model_v2.pdb' % row.Uniprot
        uniprot.loc[i,'url'] = url
    else:
        url = None
        
        
        
print(uniprot)
uniprot.to_csv('AF_initial/1514_AF_url.csv',index=False)


for i,row in uniprot.iterrows():
    pdb = row.PDB_ID
    url_exists = not pd.Series(row.url).isna()[0]
    if url_exists:
        file = check_file(f'AF_pdb_files/{pdb}.pdb')
        filename = '' + file
        # filename = 'AF_pdb_files/AF_%s.pdb' % row.PDB_ID
        try:
            r = requests.get(row.url)
            with open(filename,'w+') as f:
                f.write(r.text)
        except:
            continue

     PDB_ID Uniprot                                           Sequence  \
0      1a12  P18754  MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...   
1      1a1x  P56278  MAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...   
2      1a62  P0AG30  MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...   
3      1a9x  P00968  MPKRTDIKSILILGAGPIVIGQACEFDYSGAQACKALREEGYRVIL...   
4      1a9x  P0A6F1  MIKSALLVLEDGTQFHGRAIGATGSAVGEVVFNTSMTGYQEILTDP...   
...     ...     ...                                                ...   
1510   1mtp  Q47NK3  MSGGFLRDDHLEFALHLHRRLAEAVPDGEVIWSPYSVACALGVLAA...   
1511   1n13  Q57764  MNAEINPLHAYFKLPNTVSLVAGSSEGETPLNAFDGALLNAGIGNV...   
1512   1qd6  P0A921  MRTLQGWLLPVFMLPMAVYAQEATVKEVHDAPAVRGSIIANMLQEH...   
1513   1qqp  P03305  MNTTDCFIALVQAIREIKALFLSRTTGKMELTLYNGEKKTFYSRPN...   
1514   1qtn  Q14790  MDFSRNLYDIGEQLDSEDLASLKFLSLDYIPQRKQEPIKDALMLFQ...   

                                                    url  
0     https://alphafold.ebi.ac.uk/files/AF-P18754-F1.

**To run DSSP there is code on Barton group GitHub -- repo: ProIntVar (private)**

To run DSSP on all pdb files:

input struct - iterate over pdb files

output struct - iterate but get rid of .pdb and put .dssp

iterate together for DSSP reader -- problem: some pdb files are invalid -> try/except

In [4]:
# Run DSSP
os.chdir('/Users/Sophia/Dundee/dundee/data/AF')
inv = []

from prointvar.dssp import DSSPrunner, DSSPreader

dir = os.listdir('AF_pdb_files')

for file in dir:
    input_struct = os.path.join('AF_pdb_files', file)   
    x = file[:-4]
    output_dssp = os.path.join('AF_dssp_files', x + '.dssp')
    try: 
#         #Uncomment the following line to run DSSP
        DSSPrunner(inputfile=input_struct, outputfile=output_dssp).write()  


    except OSError:
        inv.append(x)
        continue
        

df_inv = pd.DataFrame(inv)
df_inv.to_csv('AF_initial/dssp_run_extra_pdbs.csv')

2022-08-25 14:28:30,303 py.warnings  WARNING  /Users/Sophia/opt/anaconda3/envs/prointvar-dssp/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

2022-08-25 14:28:30,741 py.warnings  WARNING  /Users/Sophia/opt/anaconda3/envs/prointvar-dssp/lib/python3.10/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(

DSSP could not be created due to an error:
empty protein, or no valid complete residues
DSSP could not be created due to an error:
empty protein, or no valid complete residues
DSSP could not be created d

In [5]:
# Read dssp files

# Create df with pdb's left out

data = []

dir_2 = os.listdir('AF_dssp_files')

for file in dir_2:
    try:
        pdb = file[:-5]
        output_dssp = os.path.join('AF_dssp_files', file)
        df2 = DSSPreader(inputfile=output_dssp).read(add_ss_reduced=True)
        df2.head()
        
        df2.to_csv('AF_csv_predictions/%s.csv' % pdb, index=False)
        
        df_2 = df2.iloc[:, [2,-2]]
        df_2.to_csv('AF_aa_ss/%s.csv' % pdb, index=False) 
    except:
        data.append(file)
        #print('pdb:' + file)
        continue
        

df_2 = pd.DataFrame(data)
df_2.to_csv('AF_initial/dssp_read_extra_pdbs.csv')

2022-08-25 14:30:51,019 prointvar    INFO     Parsing DSSP from lines...
2022-08-25 14:30:51,071 prointvar    INFO     DSSP added full chain...
2022-08-25 14:30:51,085 prointvar    INFO     DSSP added reduced SS...
2022-08-25 14:30:51,110 prointvar    INFO     DSSP added RSA...
2022-08-25 14:30:51,168 prointvar    INFO     Parsing DSSP from lines...
2022-08-25 14:30:51,207 prointvar    INFO     DSSP added full chain...
2022-08-25 14:30:51,219 prointvar    INFO     DSSP added reduced SS...
2022-08-25 14:30:51,240 prointvar    INFO     DSSP added RSA...
2022-08-25 14:30:51,286 prointvar    INFO     Parsing DSSP from lines...
2022-08-25 14:30:51,322 prointvar    INFO     DSSP added full chain...
2022-08-25 14:30:51,334 prointvar    INFO     DSSP added reduced SS...
2022-08-25 14:30:51,350 prointvar    INFO     DSSP added RSA...
2022-08-25 14:30:51,394 prointvar    INFO     Parsing DSSP from lines...
2022-08-25 14:30:51,426 prointvar    INFO     DSSP added full chain...
2022-08-25 14:30:51

In [10]:
# Check how many pdb, dssp and csv files there are
x_1 = []
x_2 = []
x_3 = []
x_4 = []
# os.chdir('data')
dir = os.listdir('AF_pdb_files')
dir_2 = os.listdir('AF_dssp_files')
dir_3 = os.listdir('AF_csv_predictions')
dir_4 = os.listdir('AF_aa_ss')

for file in dir:
    pdb = file[:-4]
    x_1.append(pdb)
print(len(x_1))

for file in dir_2:   
    pdb = file[3:-5]
    x_2.append(pdb)
print(len(x_2))

for file in dir_3:
    csv = file[:-4]
    x_3.append(csv)
print(len(x_3))

for file in dir_4:
    x_4.append(file)
print(len(x_4))

3030
1151
1151
1151


Do it for all:

In [7]:
dir = os.listdir('AF_aa_ss')

for file in dir:       
    path = os.path.join('AF_aa_ss', file)
    #print(path)
    pdb = file[:-4]
   # print(pdb)
    try:
        x = pd.read_csv(path)
        x_t = x.T
        y = x_t.fillna('-')
        # y.to_csv('AF_aa_ss_t/%s.csv' % pdb, index=False)
        
        y['all'] = y.apply(lambda x: ''.join(x.dropna().astype(str)),axis=1)
        z = y['all']
        
        z_t = z.T
        y_t = y.T
        
        final = y_t.iloc[-1:]
        
        # final.to_csv('AF_merged/%s.csv' % pdb, index=False)
        
        add_pdb = final.rename(index={'all':pdb} )
        
        add_pdb.to_csv('AF_merged/%s.csv' % pdb)
       
    except:
        continue

In [10]:
# Check how many files there are
x_1 = []
x_2 = []

dir = os.listdir('AF_aa_ss')
dir_2 = os.listdir('AF_merged')


for file in dir:
    pdb = file[:-4]
    x_1.append(pdb)
print(len(x_1))


for file in dir_2:
    x_2.append(file)
print(len(x_2))

1151
1151


In [24]:
# Concatenate - merge all csv files into one

os.chdir('/Users/Sophia/Dundee/dundee/data/AF/AF_merged')

x_1 = []
x_2 = []

# dir = os.listdir('AF_merged')
dir = glob.glob('*.csv')

for file in dir:
    x_2.append(file)
    path = os.path.join('', file)

    try:
        x = pd.read_csv(path)

        x_1.append(x)
    except:
        x_1.append(None)
        continue
print(len(x_1))
print(len(x_2))


merged = pd.concat(x_1)
merged = merged.iloc[:, [0,1,2]]
merged.columns = ('PDB_ID', 'AA', 'SS')
merged

merged.to_csv('/Users/Sophia/Dundee/dundee/data/AF/AF_m.csv')

merged

# Can merge from terminal - use sed /cluster/gjb_lab/2524591/

1151
1151


,PDB_ID,AA,SS
0,1syx_1,MSYMLPHLHNGWQVDQAILSEEDRVVVIRFGHDWDPTCMKMDEVLY...,CCCCCCCCCCHHHHHHHHHHCCCCEEEEEEECCCCHHHHHHHHHHH...
0,3sxu,MASRGVNKVILVGNLGQDPEVRYMPNGGAVANITLATSESWRDKAT...,CCCCCCCCEEEEEEECCCCEEEECCCCCEEEEEEEEEEEEEECCCC...
0,3f42,MDFSQLGGLLDGMKKEFSQLEEKNKDTIHTSKSGGGMVSVSFNGLG...,CCHHHHHHHHHHHHHHHHHHHHHHHCCEEEEEECCCCEEEEEECCC...
0,2lig,MFNRIRVVTMLMMVLGVFALLQLVSGGLLFSSLQHNQQGFVISNEL...,CCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH...
0,2czv_1,MVGGGGVKFIEMDIRDKEAYELAKEWFDEVVVSIKFNEEVDKEKLR...,CCCCCCCCCEEEEECCHHHHHHHHHHCCEEEEEEEECCCCCHHHHH...
...,...,...,...
0,1k8k_4,MSKNTVSSARFRKVDVGEYDENKFVDEEDGGDGQAGPDEGEVDSCL...,CCCCHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHH...
0,1bx7,TQGNTCGGETCSAAQVCLKGKCVCNEVHCRIRCKYGLKKDENGCEY...,CCCCEECCEECCCCEEEECCEEEECCCCCCCCCCCCEEECCCCCEE...
0,3fau,MPRRRKNLGGNPFRKTANPKEVVVSSVASREEPTTTLPSMGETKVD...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
0,1kqp,MSMQEKIMRELHVKPSIDPKQEIEDRVNFLKQYVKKTGAKGFVLGI...,CCHHHHHHHHCCCCCCCCHHHHHHHHHHHHHHHHHHHCCCEEEEEC...


In [25]:
# Add uniprot-id
os.chdir('/Users/Sophia/Dundee/dundee/data/AF')

df_uniprot = pd.read_csv('AF_initial/uniprot-pdb_aug.csv')
df_uniprot.columns = ['PDB', 'Uniprot', 'Sequence']


dir = os.listdir('AF_merged')

# hm = merged.iloc[:,0]
merged['PDB'] = [x[:4] for x in merged['PDB_ID']]
merged

z = pd.merge(df_uniprot, merged, on= 'PDB', how = 'right')
# #print(z)      
# # z.to_csv('AF_pdb_uniprot.csv')

z.drop("Sequence", axis=1, inplace=True)
# #print(final)
# z.columns=['PDB_ID', 'UniprotID', 'Uniprot_sequence', 'AF_SS_pred']
z = z.reindex(columns = ['PDB', 'PDB_ID', 'Uniprot', 'AA', 'SS'])
z.to_csv('AF_final.csv')
z


,PDB,PDB_ID,Uniprot,AA,SS
0,1syx,1syx_1,O95400,MSYMLPHLHNGWQVDQAILSEEDRVVVIRFGHDWDPTCMKMDEVLY...,CCCCCCCCCCHHHHHHHHHHCCCCEEEEEEECCCCHHHHHHHHHHH...
1,1syx,1syx_1,P83876,MSYMLPHLHNGWQVDQAILSEEDRVVVIRFGHDWDPTCMKMDEVLY...,CCCCCCCCCCHHHHHHHHHHCCCCEEEEEEECCCCHHHHHHHHHHH...
2,3sxu,3sxu,P0AGE0,MASRGVNKVILVGNLGQDPEVRYMPNGGAVANITLATSESWRDKAT...,CCCCCCCCEEEEEEECCCCEEEECCCCCEEEEEEEEEEEEEECCCC...
3,3sxu,3sxu,P28632,MASRGVNKVILVGNLGQDPEVRYMPNGGAVANITLATSESWRDKAT...,CCCCCCCCEEEEEEECCCCEEEECCCCCEEEEEEEEEEEEEECCCC...
4,3sxu,3sxu,P28905,MASRGVNKVILVGNLGQDPEVRYMPNGGAVANITLATSESWRDKAT...,CCCCCCCCEEEEEEECCCCEEEECCCCCEEEEEEEEEEEEEECCCC...
...,...,...,...,...,...
3300,1k8k,1k8k_4,Q58CQ2,MSKNTVSSARFRKVDVGEYDENKFVDEEDGGDGQAGPDEGEVDSCL...,CCCCHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHH...
3301,1bx7,1bx7,P80302,TQGNTCGGETCSAAQVCLKGKCVCNEVHCRIRCKYGLKKDENGCEY...,CCCCEECCEECCCCEEEECCEEEECCCCCCCCCCCCEEECCCCCEE...
3302,3fau,3fau,Q86UW6,MPRRRKNLGGNPFRKTANPKEVVVSSVASREEPTTTLPSMGETKVD...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
3303,1kqp,1kqp,P08164,MSMQEKIMRELHVKPSIDPKQEIEDRVNFLKQYVKKTGAKGFVLGI...,CCHHHHHHHHCCCCCCCCHHHHHHHHHHHHHHHHHHHCCCEEEEEC...
